In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from scipy.signal import find_peaks
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, chi2, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from imblearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from scipy import stats

Using TensorFlow backend.


In [2]:
def support_vector_machine(x_train_filename, x_test_filename, y_train_filename, y_test_filename):

    # Load dataset
    df_X_train = pd.read_csv(x_train_filename, header=None)
    df_Y_train = pd.read_csv(y_train_filename, header=None)

    df_X_test = pd.read_csv(x_test_filename, header=None)
    df_Y_test = pd.read_csv(y_test_filename, header=None)

    X_train = np.array(df_X_train.values)
    Y_train = np.array(df_Y_train.values)

    X_test = np.array(df_X_test.values)
    Y_test = np.array(df_Y_test.values)
    
    def basic_pipeline(X, y, X_test):
        
        line = Pipeline([
            ('var', VarianceThreshold()),
            ('scale', StandardScaler()),
            ('clas', svm.SVC(decision_function_shape='ovr',class_weight='balanced'))
        ])


        parameters = {"clas__C": [1, 2, 4, 8, 16],
                      "clas__gamma": [0.01, 0.1, 1]
                     }
        grid = GridSearchCV(line, parameters, scoring='balanced_accuracy', cv=5, n_jobs=-1, verbose=1)
        grid.fit(X, y)
        
        return grid, grid.predict(X_test)
    
    grid_small, y_hat = basic_pipeline(X_train, Y_train, X_test)
    
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(Y_test, y_hat)
    accuracy_str = 'Accuracy: %f' % accuracy 
    print(accuracy_str)
    # precision tp / (tp + fp)
    precision = precision_score(Y_test, y_hat, labels=range(5), average='weighted')
    precision_str = 'Precision: %f' % precision 
    print(precision_str)
    # recall: tp / (tp + fn)
    recall = recall_score(Y_test, y_hat, labels=range(5), average='weighted')
    recall_str = 'Recall: %f' % recall
    print(recall_str)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(Y_test, y_hat, labels=range(5), average='weighted')
    f1_str = 'F1 score: %f' % f1 
    print(f1_str)
    # Calculate roc auc
    roc = roc_auc_score(Y_test, y_hat)
    roc_str = 'ROC: %f' % roc
    print(roc_str)
    #Confusion matrix
    cm = confusion_matrix(Y_test, y_hat)
    print(cm)
    
    # Write output results in file
    out = open("output_svm.txt","w") 
    out.write("Result metrics:" + "\n")
    out.write(accuracy_str + "\n")
    out.write(precision_str + "\n")
    out.write(recall_str + "\n")
    out.write(f1_str + "\n")
    out.close()         